##  We're going to look for data anomalies

In [1]:
#First set everthing up
import pyodbc
import graphlab as gl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import local #server information and credentials
from pullODBC import pullODBC
from pullODBCall import pullODBCall
from matplotlib.backends.backend_pdf import PdfPages
from __future__ import division # convert integer division to floating point
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['font.size'] = 7
plt.tight_layout;
plt.rc('xtick', labelsize=7) 
plt.rc('ytick', labelsize=7) 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Section A: Extracting Voter History Data for each Registered Voter  to find data anomalies

In [2]:
#Set up the fields that I need to pull from the database 
fields = '["ncid"], ["election_lbl"], ["county_desc"], ["pct_label"],["voted_county_desc"], ["voting_method"]'
print fields

["ncid"], ["election_lbl"], ["county_desc"], ["pct_label"],["voted_county_desc"], ["voting_method"]


Change the database to 'history' and the table to 'ncvhis_Statewide'

In [3]:
local.database = 'history'
local.table = 'ncvhis_Statewide'

In [4]:
history=pullODBCall(local.driver,
                      local.server,
                      local.database,
                      local.username,
                      local.password,
                      local.table,
                      fields)

Get rid of all the double quotes from the dataframe

In [5]:
history.columns = history.columns.str.strip('"')
for i, col in enumerate(history.columns):
    history.iloc[:, i] = history.iloc[:, i].str.replace('"', '')
print history.head(n=15)

        ncid election_lbl county_desc pct_label voted_county_desc  \
0    AA10737   11/03/2015    ALAMANCE       124          ALAMANCE   
1    AA10737   11/05/2013    ALAMANCE       124          ALAMANCE   
2    AA10737   05/06/2008    ALAMANCE       124          ALAMANCE   
3    AA10737   11/04/2008    ALAMANCE       124          ALAMANCE   
4    AA10737   10/09/2007    ALAMANCE       124          ALAMANCE   
5    AA10737   11/08/2011    ALAMANCE       124          ALAMANCE   
6    AA10737   11/04/2014    ALAMANCE       124          ALAMANCE   
7   AA107373   11/04/2014    ALAMANCE       064          ALAMANCE   
8   AA107373   11/06/2007    ALAMANCE       12E          ALAMANCE   
9   AA107373   11/02/2010    ALAMANCE       064          ALAMANCE   
10  AA107373   11/08/2016    ALAMANCE       064          ALAMANCE   
11  AA107373   03/15/2016    ALAMANCE       064          ALAMANCE   
12  AA107373   11/04/2008    ALAMANCE       064          ALAMANCE   
13  AA107373   11/06/2012    ALAMA

In [6]:
#Save the total number of rows to raw_len and print. 
raw_len=len(history)
print "The number of rows pulled from the database is " + str(raw_len)

The number of rows pulled from the database is 32482836


Create a generic function to change blank fields to NaN for all categorical fields 

In [7]:
def fix_blanks_NaN(x):
    if x == "":
        return np.nan
    elif x == " ":
        return np.nan
    else:
        return x

In [8]:
history['election_lbl'] = history['election_lbl'].apply(fix_blanks_NaN) # change blanks to NaN
history['ncid'] = history['ncid'].apply(fix_blanks_NaN) # etc.
history['county_desc'] = history['county_desc'].apply(fix_blanks_NaN) # etc.

Clean the data by dropping rows with missing data and print a summary

In [9]:
history.dropna(inplace=True, how='any') #drop all rows with NaN
clean_len=len(history) #how many rows to I have left?
difference = raw_len - clean_len #how many rows have I dropped?
fraction = float(difference)/float(raw_len) #what fraction of rows have I dropped?
print "The number of rows I have dropped is " + str(difference) 
print "The number of rows remaining in the database is " + str(clean_len)
print "The precentage of rows that I have dropped is: " + str('{:.3%}'.format(fraction))

The number of rows I have dropped is 0
The number of rows remaining in the database is 32482836
The precentage of rows that I have dropped is: 0.000%


Now we need to discard all elections not in November, since we only want to analyze general elections

In [10]:
history = history[history['election_lbl'].astype(str).str[0:2]=='11']

We need to flatten the database to a single row per NCID and massage the data

In [11]:
elections_by_ncid = pd.pivot_table(history[['ncid','election_lbl']], index=['ncid'],  
                                 aggfunc=len, columns='election_lbl', margins = False) # create a pivot table
elections_flattened = pd.DataFrame(elections_by_ncid.to_records()) # convert the pivot table to a dataframe
cols = elections_flattened.columns # get an index object with all the column names
cols = cols.delete(0) # remove the first column name 'ncid' since I don't want to rename this
elections_flattened.fillna(value=0, method=None, axis=None, inplace=True, limit=None, downcast=None) # replace NaN with 0
elections_flattened[cols] = elections_flattened[cols].astype(int)# convert values to integers
for col in cols: # rename each election column to only the election year rather than the full date
    elections_flattened.rename(index=str, inplace=True, columns={col : col[-4:]})
elections_flattened = elections_flattened.reindex_axis(sorted(elections_flattened.columns), axis=1) # sort the columns by year

print elections_flattened.head()

   2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  2018      ncid
0     0     1     0     0     0     0     0     0     0     1     0  AA100000
1     0     1     0     1     0     1     0     1     0     1     0  AA100006
2     0     1     0     1     1     1     0     1     0     1     0  AA100007
3     0     1     0     0     0     1     0     0     0     0     0  AA100008
4     0     1     0     1     0     1     0     1     0     1     0  AA100009


Let's examine some anomolies in the voter history data.  Notice above that there is a column for 2018.  This means there is a record in the data for 2018.  Let's first find out which NCID is assocated with this record.

In [12]:
del elections_by_ncid # This is not used anymore sol let's delete it to free up some memory
pd.set_option('display.height', 5000)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print elections_flattened[elections_flattened['2018']==1]

height has been deprecated.

         2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  2018       ncid
3355290     0     0     0     0     0     0     0     0     0     0     1  CW1008993


We've identified NCID CW1008993.  Let's look at the entire November voting history for this NCID from 2007 thru 2016.

In [13]:
voter_2018 = history[history['ncid']== 'CW1008993']

In [14]:
print voter_2018

               ncid election_lbl  county_desc pct_label voted_county_desc voting_method
16586046  CW1008993   11/06/2018  MECKLENBURG       061       MECKLENBURG     IN-PERSON


Apparently, the person associated with NCID CW1008993 only has a single record for 2018.  Since November 6th was election day in both 2007 and 2012, this record may be a typo from that year.

Let's now look at instances where more than one voting record exists for an ncid for a November election.  First, we see how many NCIDs are associated with multiple voting records for each year

In [15]:
for year in ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']:
        print len(elections_flattened[elections_flattened[year]>1])

31
521
11
142
10
239
8
49
1
67


Considering the fact that we have over 32M voting records, this is a relatively small number.  However, these cases need investigation.  Since the voter history file has over 32M records, it's difficult to reshape and analyze in it's raw form.  Therefore, we need to create a subset of the voter history file for analysis.  To do this, we'll first identify any and all ncids that have more than one voting record for a specific November election.  We will then create a subset of the voter histroy file only containing records with these ncids.

Lets group the data by NCID (only for NCIDs associated with multiple voting records) and generate a report for all of these NCIDs.

In [16]:
multiple_records = elections_flattened[(elections_flattened['2007']>1) | (elections_flattened['2008']>1)\
|(elections_flattened['2009']>1) | (elections_flattened['2010']>1)\
|(elections_flattened['2011']>1) | (elections_flattened['2012']>1)\
|(elections_flattened['2013']>1) | (elections_flattened['2014']>1)\
|(elections_flattened['2015']>1) | (elections_flattened['2016']>1)]

In [17]:
print len(multiple_records)

825


It looks like a total of 825 NCIDs have multiple records for November elections from 2007 thru 2016.  We need to select records from the voter history data corresponding only to these NCIDs.  First create an iterable series of NCIDs from multiple_records.

In [18]:
multiple_series = multiple_records['ncid']

Next, let's subset out the history dataframe to include only those records associated with voters having multiple records in any year

In [19]:
history_subset = history[history['ncid'].isin(multiple_series)]
print len(history_subset)

4149


We have 4149 records.  Now let's group by NICD and election_lbl

In [20]:
### First stip out blanks from election_lbl
history_subset['election_lbl']= history_subset['election_lbl'].map(lambda x: x.strip())
history_subset = history_subset.sort_values(by = ['ncid', 'election_lbl'], ascending=[1, 1])
report = history_subset.set_index(['ncid', 'election_lbl', 'county_desc'])
print len(report)
print report.head(n=15)

4149
                                  pct_label voted_county_desc     voting_method
ncid     election_lbl county_desc                                              
AA131256 11/04/2008   GUILFORD          03N          ALAMANCE       PROVISIONAL
                      GUILFORD          03N          ALAMANCE       PROVISIONAL
AA173882 11/04/2008   ALAMANCE          12N          ALAMANCE         IN-PERSON
                      ALAMANCE          12N          ALAMANCE         IN-PERSON
         11/06/2012   ALAMANCE          127          ALAMANCE  ABSENTEE ONESTOP
         11/08/2016   ALAMANCE          03N          ALAMANCE  ABSENTEE ONESTOP
AB14598  11/02/2010   ANSON              B2         ALEXANDER         IN-PERSON
         11/04/2008   ANSON              B2         ALEXANDER         IN-PERSON
         11/04/2014   ANSON            LILE             ANSON  ABSENTEE ONESTOP
         11/06/2012   ANSON              B2         ALEXANDER         IN-PERSON
                      ANSON        

C:\Users\scott\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


This list is too long to analyze.  Let's only select the nicd / election_lbl combinations where there is more than one "vote"

In [21]:
# First, reset the indexs and create a groupby object
reset_report = report.reset_index()# Reset the index
num_voting_records = reset_report.groupby(['ncid', 'election_lbl'])['county_desc'].count() # Create a groupBy object with counts of number of "votes"
multiple = num_voting_records[num_voting_records >1] # Only keep entries with more than 1 "vote"

The series "multiple" contains all the indexes ( ncid / election_lbl combinations) where there was more than 1 "vote" recored.  Let's use this series to iterate on the "reset_report" dataframe to only select instances of more than one vote

In [22]:
index_list = list() # Create an empty list for the indexes to keep

for row in multiple.index.values:
    ncid = row[0]
    election_lbl = row[1]
    selected_row_indices = reset_report[(reset_report['ncid']== ncid) & (reset_report['election_lbl']== election_lbl)].index[0:2]
    index_list.append(selected_row_indices)
    

In [23]:
selected = [element for tupl in index_list for element in tupl]
new_report = reset_report[reset_report.index.isin(selected)]
#To simplify report, let's delete county_desc since this is only the current county where the voter is registered.  We will only focus on voted_county_desc
new_report.drop('county_desc',axis=1, inplace=True)
print len(new_report)/2
new_report =  new_report.set_index(['ncid', 'election_lbl', 'voted_county_desc'])

1079.0


C:\Users\scott\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Let me know save this to a text file

In [24]:
%store new_report > multiple_voting_records_june2017_snapshot.txt

Writing 'new_report' (DataFrame) to file 'multiple_voting_records_june2017_snapshot.txt'.


Clean up old dataframes

In [25]:
del history
del elections_flattened

### Section B: Extracting Voter Demographics to find data anomalies

First change the database and table names back to get registration data

In [26]:
local.database = 'registration'
local.table = 'ncvoter_Statewide'

We now do the standard extraction and cleaning.  I've excluded comments since task has been done many times

In [27]:
#Set up the fields that I need to pull from the database 
fields = '["birth_age"], ["gender_code"], ["race_code"], ["ethnic_code"],["party_cd"], ["status_cd"], ["zip_code"], ["birth_state"], ["drivers_lic"], ["county_desc"], ["registr_dt"],["ncid"],["precinct_abbrv"]'
print fields

["birth_age"], ["gender_code"], ["race_code"], ["ethnic_code"],["party_cd"], ["status_cd"], ["zip_code"], ["birth_state"], ["drivers_lic"], ["county_desc"], ["registr_dt"],["ncid"],["precinct_abbrv"]


In [28]:
condition1 = '["status_cd"] = ' + local.single_double_quote + 'A' + local.double_single_quote
condition2 = '["status_cd"] = ' + local.single_double_quote + 'I' + local.double_single_quote
condition3 = '["county_desc"] = ' + local.single_double_quote + 'Chatham' + local.double_single_quote
condition = "(" + condition1 + " OR " + condition2 + ")"  # This line will be run on Azure
#condition = "(" + condition1 + " OR " + condition2 + ")" + " AND " + condition3# This line will be run on local
print condition

(["status_cd"] = '"A"' OR ["status_cd"] = '"I"')


In [29]:
demographics=pullODBC(local.driver,
                      local.server,
                      local.database,
                      local.username,
                      local.password,
                      local.table,
                      fields,
                      condition)

Get rid of all the double quotes from the dataframe

In [30]:
demographics.columns = demographics.columns.str.strip('"')
for i, col in enumerate(demographics.columns):
    demographics.iloc[:, i] = demographics.iloc[:, i].str.replace('"', '')
print demographics.head()

  birth_age gender_code race_code ethnic_code party_cd status_cd zip_code birth_state drivers_lic county_desc  registr_dt      ncid precinct_abbrv
0        79           M         W          NL      DEM         A    27253          NC           N    ALAMANCE  01/19/1959   AA34771             07
1        60           M         W          NL      REP         A    27215          NC           Y    ALAMANCE  04/09/1984   AA53687            12W
2        43           M         W          NL      REP         A    27302          NC           Y    ALAMANCE  04/02/1992   AA80986            10S
3        55           M         W          NL      REP         A    27215          NC           Y    ALAMANCE  09/15/2003  AA131587            03S
4        58           F         W          NL      DEM         I    27258          NC           Y    ALAMANCE  05/19/2006  AA146650             13


In [31]:
#Save the total number of rows to raw_len and print. 
raw_len=len(demographics)
print "The number of rows pulled from the database is " + str(raw_len)

The number of rows pulled from the database is 6774419


Create a generic function to change blank fields to 'None' for some categorical fields 

In [32]:
def fix_blanks(x):
    if x == "":
        return "None"
    elif x == " ":
        return "None"
    else:
        return x

Create a generic function to change blank fields to NaN for all categorical fields 

In [33]:
def fix_blanks_NaN(x):
    if x == "":
        return np.nan
    elif x == " ":
        return np.nan
    else:
        return x

In [34]:
demographics['birth_state'] = demographics['birth_state'].apply(fix_blanks) # change blanks to "None" in birth_state
demographics['gender_code'] = demographics['gender_code'].apply(fix_blanks_NaN) # change blanks to NaN in gender_code
demographics['race_code'] = demographics['race_code'].apply(fix_blanks_NaN) # etc.
demographics['ethnic_code'] = demographics['ethnic_code'].apply(fix_blanks_NaN) 
demographics['zip_code'] = demographics['zip_code'].apply(fix_blanks_NaN)
demographics['birth_state'] = demographics['birth_state'].apply(fix_blanks_NaN) 
demographics['drivers_lic'] = demographics['drivers_lic'].apply(fix_blanks_NaN) 
demographics['county_desc'] = demographics['county_desc'].apply(fix_blanks_NaN) 
demographics['registr_dt'] = demographics['registr_dt'].apply(fix_blanks_NaN)
demographics['ncid'] = demographics['ncid'].apply(fix_blanks_NaN)
demographics['precinct_abbrv'] = demographics['precinct_abbrv'].apply(fix_blanks_NaN)

In [35]:
demographics.dropna(inplace=True, how='any') #drop all rows with NaN
clean_len=len(demographics) #how many rows to I have left?
difference = raw_len - clean_len #how many rows have I dropped?
fraction = float(difference)/float(raw_len) #what fraction of rows have I dropped?
print "The number of rows I have dropped is " + str(difference) 
print "The number of rows remaining in the database is " + str(clean_len)
print "The precentage of rows that I have dropped is: " + str('{:.3%}'.format(fraction))

The number of rows I have dropped is 3156
The number of rows remaining in the database is 6771263
The precentage of rows that I have dropped is: 0.047%


In [36]:
def convert_to_float(x): # function to convert a timedelta object to a float of years
    return x.total_seconds()/(60*60*24*365.2422)

In [37]:
demographics['birth_age'] = demographics['birth_age'].astype('float64')
demographics['registr_dt'] = pd.to_datetime(demographics['registr_dt'], format='%m/%d/%Y')
demographics['years_since_registration'] = pd.to_datetime('today') - demographics['registr_dt'] # get days since registration as a timedelta object
demographics['years_since_registration'] = demographics['years_since_registration'].apply(convert_to_float) # convert to years in floating point


Frist, let's look at voters over 115 years of age

In [38]:
print len(demographics[demographics['birth_age']> 115])
print demographics[demographics['birth_age']> 115][['ncid', 'birth_age']].head(n=20)

5201
           ncid  birth_age
40378   AA12753      117.0
65502   AA15391      117.0
65807   AG16763      116.0
65809   AG16764      116.0
66346   AA64367      117.0
78704   AA24092      117.0
101378   AB8204      117.0
102552   AB4557      117.0
105985   AB2589      117.0
114305   AG5279      116.0
114715   AG9385      116.0
126226   AG9308      116.0
132188  AG19708      116.0
139591  AG11778      116.0
139627  AG14826      116.0
140097  AD12103      116.0
144034   AB2994      117.0
147012  AG11592      116.0
154927   AB2684      117.0
171588   AB6855      117.0


These are obviously errors.  Let's create a report sorting these folks by county and storing this in a report.

In [39]:
subset = demographics[demographics['birth_age']> 115][['county_desc', 'ncid', 'birth_age','registr_dt']]
subset = subset.sort_values(by = ['county_desc','birth_age'], ascending=True)
subset = subset.set_index(['county_desc','birth_age']) #sort by county
%store subset > age_errors_june2017_snapshot.txt

Writing 'subset' (DataFrame) to file 'age_errors_june2017_snapshot.txt'.


Let's now create a report with counts by county

In [40]:
# First, reset the indexs and create a groupby object
subset = subset.reset_index()# Reset the index
num_invalid_age = subset.groupby(['county_desc'])['county_desc'].count() # Create a groupBy object with counts of number of voters with birth_age > 115
num_invalid_age.nlargest(100)# Sort by number of voters

county_desc
GUILFORD       2245
FORSYTH        1489
CUMBERLAND      459
RANDOLPH        266
HENDERSON       185
HALIFAX         153
JOHNSTON         84
SCOTLAND         61
PITT             51
BEAUFORT         31
RICHMOND         21
GRANVILLE        13
SAMPSON          12
ALEXANDER         9
WAYNE             9
EDGECOMBE         8
BRUNSWICK         7
VANCE             6
NORTHAMPTON       6
YADKIN            6
CHEROKEE          5
ROBESON           5
MARTIN            5
ORANGE            4
MCDOWELL          4
ALAMANCE          4
GATES             4
HARNETT           3
COLUMBUS          3
HERTFORD          3
BERTIE            3
ROWAN             3
MONTGOMERY        3
NASH              3
BLADEN            2
WILSON            2
CARTERET          2
PERSON            2
CHATHAM           2
FRANKLIN          2
NEW HANOVER       2
CURRITUCK         2
ONSLOW            1
CABARRUS          1
BURKE             1
MITCHELL          1
MECKLENBURG       1
BUNCOMBE          1
STANLY            1
DAVIDSON

It looks like six counties make up the bulk of the problem records: GUILFORD, FORSYTH, CUMBERLAND, RANDOLPH, HENDERSON, and HALIFAX      

Next, let's look at voters that have been registered for more than 100 years

In [41]:
print len(demographics[demographics['years_since_registration']> 100])
print demographics[demographics['years_since_registration']> 100][['ncid', 'registr_dt']].head(n=15)

10342
           ncid registr_dt
1158     AA3305 1900-01-01
2487     AA6313 1900-01-01
2510     AA6336 1900-01-01
2870   AA110542 1900-01-01
2982    AA36629 1900-01-01
2983    AA36630 1900-01-01
3058    AA36668 1900-01-01
3848     AA9057 1900-01-01
3893     AA9089 1900-01-01
5211    AA11828 1900-01-01
6553    AA13983 1900-01-01
9733     AA6556 1900-01-01
10107   AA27501 1900-01-01
12467   AA12034 1900-01-01
12471   AA12039 1900-01-01


These are obviously errors.  Let's create a report sorting these folks by county and storing this in a report.

In [42]:
subset = demographics[demographics['years_since_registration']> 100][['county_desc', 'ncid', 'registr_dt', 'birth_age']]
subset = subset.sort_values(by = ['county_desc','registr_dt'], ascending=True)
subset = subset.set_index(['county_desc','registr_dt']) #sort by county
%store subset > registration_errors_june2017_snapshot.txt

Writing 'subset' (DataFrame) to file 'registration_errors_june2017_snapshot.txt'.


Let's now create a report with counts by county

In [43]:
# First, reset the indexs and create a groupby object
subset = subset.reset_index()# Reset the index
num_invalid_registration = subset.groupby(['county_desc'])['county_desc'].count() # Create a groupBy object with counts of number of voters with invalid registration dates
num_invalid_registration.nlargest(100)# Sort by number of voters

county_desc
CUMBERLAND     8381
MCDOWELL        784
PERSON          191
ALAMANCE        139
DUPLIN          124
HALIFAX         101
WARREN           53
DURHAM           51
CATAWBA          42
BUNCOMBE         34
CHATHAM          33
ORANGE           28
IREDELL          27
NASH             15
SCOTLAND         14
BURKE            14
RANDOLPH         14
GRANVILLE        13
CLAY             13
MECKLENBURG      13
MONTGOMERY       13
ROBESON          12
STANLY           11
UNION            11
CLEVELAND        11
CHEROKEE         11
YADKIN           10
HOKE             10
ROWAN             9
MADISON           9
FORSYTH           8
CARTERET          8
PITT              7
YANCEY            7
FRANKLIN          7
BRUNSWICK         7
EDGECOMBE         7
ROCKINGHAM        6
HENDERSON         6
BERTIE            5
CURRITUCK         5
JOHNSTON          5
SAMPSON           4
HARNETT           4
VANCE             4
CASWELL           4
SURRY             3
ASHE              3
CALDWELL          3
GUILFORD

It looks like six counties make up the bulk of the problem records: CUMBERLAND, MCDOWELL, PERSON, ALAMANCE, DUPLIN, and HALIFAX            